#RO Project#

1. Read instances' data from file
2. Initialize the parameters read from file
3. n = number of customers
4. c = matrix of costs
5. ml = max load of the vehicle
6. l = linehaul values vector
7. b = backhaul values vector
8. Initialize the matrix s of savings 
9. for i = 1:n
    1. for j = 1:n
        1. If i == j
            1. s_ij = +inf
        2. else
            1. s_ij = c_i0 + c_0j - c_ij
10. Routes = list of empty routes
11. for i = 1:n
    1. Routes_i = (0,i,0)
12. Order the savings s in a non increasing fashion
13. for each route in Routes:
    1. while this route can be feasibly (respecting the constraints) merged further:
	   1.determine the first saving s_ki or s_jl in the ordered list that can feasibly be used to merge the current route with another route containing either the arc (k, 0) or (0, l). 
       	   2. merge the two routes.



In [2]:
import math
import numpy as np
import sys
import os
import time

In [3]:
def euclidean_distance2D(point1, point2):
    
    x1, y1 = point1
    x2, y2 = point2
    
    return (math.sqrt((x1-x2)**2 + (y1-y2)**2))

In [4]:
def is_route_valid (route_to_check):
    
    is_route_possible = True
    cost_of_possible_route_linehaul = 0
    cost_of_possible_route_backhaul = 0
    i = 0
    
    for node_touched in route_to_check[1:-1]:
        
        if linehaul_vector[node_touched] == 0:#linehaul_vector is 0 if a node is not a linehaul node, so we have to go to the next for
            break
        else:
            cost_of_possible_route_linehaul += linehaul_vector[node_touched]
            i+=1
            
    starting_point = i
    if i < len(route_to_check[1:-1]):
        for node_touched in route_to_check[starting_point+1:-1]:
            
            if backhaul_vector[node_touched] == 0 and i != len(route_to_check): #we are trying to linehaul AFTER we started backhauling... constraint broken:
                is_route_possible = False
                #print(i, backhaul_vector[node_touched], node_touched)
            else:
                cost_of_possible_route_backhaul += backhaul_vector[node_touched]
                i+=1
                
        
    if (cost_of_possible_route_backhaul > capacity or cost_of_possible_route_linehaul > capacity):
        is_route_possible = False
        
    if (cost_of_possible_route_linehaul == 0): # we are only backhauling
        is_route_possible = False
        
        
    return is_route_possible

In [5]:
def is_route_merge_valid(id_route1, id_route2):
    
    """First, try if you can merge it with the end of route1 to end of route2"""
        
    possible_route = list( routes[id_route1][:-1] + routes[id_route2][1:]) #the merge itself is just skipping the left and right
    
    is_right_possible = is_route_valid(possible_route)
    """Now the other possible route."""
        
    possible_route = list( routes[id_route2][:-1] + routes[id_route1][1:]) #the merge itself is just skipping the left and right
    
    is_left_possible = is_route_valid(possible_route)
    
    #print(is_left_possible, is_right_possible)
        
    if not is_left_possible and not is_right_possible:
        return ('')
    elif is_left_possible and is_right_possible:
        return ('lr')
    elif is_left_possible:
        return 'l'
    else:
        return 'r'
        
                

In [6]:
def there_is_a_possible_merge(i, j):
    
    
    for id1, route1 in enumerate(routes):
        
        for id2, route2 in enumerate(routes):
            
            if route1 == (-1, -1, -1) or route2 == (-1, -1, -1):
                continue
                
            if (id1 != id2):
                
                if (route1[1] == j and route2[-2] == i):
                    
                    return (id1, id2)
    
    return -1
                
    

In [7]:
def merge_two_routes(id_route, found_id, direction_of_merge):

    if id_route== found_id:
        return
    
    tmp_list1 = list(routes[id_route])
    tmp_list2 = list(routes[found_id])
        
    tmp_list1.pop(0)
    tmp_list1.pop(-1)
    tmp_list2.pop(0)
    tmp_list2.pop(-1)
    
    if direction_of_merge == 'l':
        
        routes[id_route] = tuple([0]+tmp_list2+tmp_list1+[0])
    
    if direction_of_merge == 'r':
        
        routes[id_route] = tuple([0]+tmp_list1+tmp_list2+[0])
    
    routes[found_id] = (-1,-1,-1)




In [8]:
def find_best_relocate():
    
    possible_changes = []
    
    for id_route1, route1 in enumerate(routes):
        
        for id_item_to_move, item_to_move in enumerate(route1[1:-1]): # remember that id is actually +1 if considered in the full route
            
            for id_route2, route2 in enumerate(routes):
                #print('route1:', route1)
                #print('item_to_move:', item_to_move)
                #print('route2:', route2)
                
                if (id_route1 == id_route2):
                    temp_route = list(route1[:id_item_to_move+1] + route1[id_item_to_move+2:])
                else:
                    temp_route = route2
                
                for id_moved, item in enumerate(temp_route[1:]): # just as above, careful with id
                    possible_change = list([0] + list(temp_route[1:id_moved+1]) + [item_to_move] + list(temp_route[id_moved+1:]))
                    if (is_route_valid(possible_change)): #and is_route_valid (list(route1[:id_item_to_move+1] + route1[id_item_to_move+2:]))):
                        #print(possible_change)
                        #print(list([0] + list(temp_route[1:id_moved+1]) + [item_to_move] + list(temp_route[id_moved+1:])))
                        #print(id_route1, id_route2, id_item_to_move, id_moved)
                        possible_changes += [(possible_change, item_to_move, id_route1, id_route2, id_item_to_move, id_moved)]
                        
                        
    most_improvement = 0
    best_change = ()

    for id_change, change in enumerate(possible_changes):

        cost_of_old_routes = 0
        cost_of_new_routes = 0
        (possible_change, item_to_move, id_route1, id_route2, id_item_to_move, id_moved) = change
        
        if (id_route1 == id_route2):
            
            for idx, item in enumerate(routes[id_route1][:-1]):
                
                cost_of_old_routes+=cost_matrix[item, routes[id_route1][idx+1]]

            new_route_1 = []
            new_route_2 = possible_change

            for idx, item in enumerate(new_route_2[:-1]):

                cost_of_new_routes+=cost_matrix[item, new_route_2[idx+1]]
        
        else:
            
        
            for idx, item in enumerate(routes[id_route1][:-1]):

                cost_of_old_routes+=cost_matrix[item, routes[id_route1][idx+1]]

            for idx, item in enumerate(routes[id_route2][:-1]):

                cost_of_old_routes+=cost_matrix[item, routes[id_route2][idx+1]]


            new_route_1 = list(routes[id_route1][:id_item_to_move+1] + routes[id_route1][id_item_to_move+2:])
            new_route_2 = possible_change

            for idx, item in enumerate(new_route_1[:-1]):

                cost_of_new_routes+=cost_matrix[item, new_route_1[idx+1]]

            for idx, item in enumerate(new_route_2[:-1]):

                cost_of_new_routes+=cost_matrix[item, new_route_2[idx+1]]

        #print('cost of old routes:', cost_of_old_routes)
        #print('cost of new routes:', cost_of_new_routes)
        
        if (cost_of_new_routes < cost_of_old_routes):
            #print(cost_of_old_routes - cost_of_new_routes)

            if cost_of_old_routes - cost_of_new_routes > most_improvement:
                most_improvement = cost_of_old_routes - cost_of_new_routes
                best_change = ( most_improvement, (id_route1, new_route_1) , (id_route2, new_route_2) )
                #print(most_improvement)
                
                
    
    #print(most_improvement)
    return best_change
            
        

In [9]:
def find_best_exchange():
    
    possible_changes = []
    
    for id_route1, route1 in enumerate(routes):
        
        for id_item_to_exchange1, item_to_exchange1 in enumerate(route1[1:-1]): # remember that id is actually +1 if considered in the full route
            
            for id_route2, route2 in enumerate(routes):
                
                if(id_route1 > id_route2):
                    continue
                        
                
                for id_item_to_exchange2, item_to_exchange2 in enumerate(route2[1:-1]):
                    
                    possible_change1 = list(route1)
                    possible_change2 = list(route2)
                    
                    if id_route1 == id_route2:
                        possible_change1[id_item_to_exchange1], possible_change1[id_item_to_exchange2] = possible_change1[id_item_to_exchange2], possible_change1[id_item_to_exchange1]
                        possible_change2[id_item_to_exchange1], possible_change2[id_item_to_exchange2] = possible_change2[id_item_to_exchange2], possible_change2[id_item_to_exchange1]

                    
                    else:
                        possible_change1[id_item_to_exchange1+1] = item_to_exchange2
                        possible_change2[id_item_to_exchange2+1] = item_to_exchange1
                    #print((possible_change1, possible_change2))


                
                    if (is_route_valid(possible_change1) and is_route_valid(possible_change2)): #and is_route_valid (list(route1[:id_item_to_move+1] + route1[id_item_to_move+2:]))):


                        #print(possible_change)
                        #print(list([0] + list(temp_route[1:id_moved+1]) + [item_to_move] + list(temp_route[id_moved+1:])))
                        #print(id_route1, id_route2, id_item_to_move, id_moved)
                        possible_changes += [(possible_change1, possible_change2, item_to_exchange1, item_to_exchange2, id_route1, id_route2, id_item_to_exchange1, id_item_to_exchange2)]

                        
    most_improvement = 0
    best_change = ()

    for id_change, change in enumerate(possible_changes):

        cost_of_old_routes = 0
        cost_of_new_routes = 0
        (possible_change1, possible_change2, item_to_exchange1, item_to_exchange2, id_route1, id_route2, id_item_to_exchange1, id_item_to_exchange2) = change
        
        if (id_route1 == id_route2):
            
            for idx, item in enumerate(routes[id_route1][:-1]):
                
                cost_of_old_routes+=cost_matrix[item, routes[id_route1][idx+1]]

            new_route_1 = []
            new_route_2 = possible_change2

            for idx, item in enumerate(new_route_2[:-1]):

                cost_of_new_routes+=cost_matrix[item, new_route_2[idx+1]]
        
        else:
            
        
            for idx, item in enumerate(routes[id_route1][:-1]):

                cost_of_old_routes+=cost_matrix[item, routes[id_route1][idx+1]]

            for idx, item in enumerate(routes[id_route2][:-1]):

                cost_of_old_routes+=cost_matrix[item, routes[id_route2][idx+1]]


            new_route_1 = possible_change1
            new_route_2 = possible_change2

            for idx, item in enumerate(new_route_1[:-1]):

                cost_of_new_routes+=cost_matrix[item, new_route_1[idx+1]]

            for idx, item in enumerate(new_route_2[:-1]):

                cost_of_new_routes+=cost_matrix[item, new_route_2[idx+1]]

        #print('cost of old routes:', cost_of_old_routes)
        #print('cost of new routes:', cost_of_new_routes)
        
        if (cost_of_new_routes < cost_of_old_routes):
            #print(cost_of_old_routes - cost_of_new_routes)

            if cost_of_old_routes - cost_of_new_routes > most_improvement:
                most_improvement = cost_of_old_routes - cost_of_new_routes
                best_change = ( most_improvement, (id_route1, new_route_1) , (id_route2, new_route_2) )
                #print(most_improvement)
                
                
    
    #print(most_improvement)
    return best_change

In [10]:
def get_cost_matrix():

    cost_matrix = []

    x_coords = [int(i.split()[0]) for i in content[3:]]
    y_coords = [int(i.split()[1]) for i in content[3:]]

    for i, (x1, y1) in enumerate(zip(x_coords, y_coords)): #could also enumerate over y_coords, the dimensions are equal

        current_row = []

        for j, (x2, y2) in enumerate(zip(x_coords, y_coords)):

            current_row += [euclidean_distance2D((x1,y1), (x2,y2))]

        cost_matrix += [current_row]
    
    cost_matrix = np.array(cost_matrix)
    return cost_matrix
        

In [11]:
def get_savings_matrix ():
    savings_matrix = np.zeros((n_of_customers + 1, n_of_customers + 1))
    for i in range(n_of_customers + 1):
        for j in range(n_of_customers + 1):
            if (i==j):
                savings_matrix[i][j] = sys.maxsize
            else:
                savings_matrix[i][j] = cost_matrix[i][0] + cost_matrix[0][j] - cost_matrix[i][j]
    return savings_matrix

In [12]:
def compute_total_cost():
    total_cost = 0

    for route in routes:

        if route != (-1, -1, -1):

            for idx, item in enumerate(route[:-1]):

                total_cost+=cost_matrix[item, route[idx+1]]
    return total_cost

In [13]:
#MERGE ROUTES BASED ON THE ALGORITHM CONSTRAINTS
def merge_routes():
    for idx, saving in enumerate(ordered_savings[n_of_customers+1:]): # 'n_customers+1' because we are not interested in the infinite savings, since their i,j pairs won't allow feasible merges
    
        i, j = saving[1]
    
        res = there_is_a_possible_merge(i,j)
    
        if res != -1:
        
            id1, id2 = res
            direction = is_route_merge_valid(id1, id2)
            if direction == 'l' or direction == 'lr':
                merge_two_routes(id1, id2, 'l')
    return routes

In [14]:
def find_best_improvement(mode):
    new_cost = 0
    previous_cost = total_cost
    threshold = 5

    while previous_cost - new_cost > threshold:

        possible_change = ()
        if mode == 'relocate':
            possible_change = find_best_relocate()
        else:
            possible_change = find_best_exchange()
        previous_cost = new_cost

        if possible_change != ():

            improvement, (old_route1_index, new_route_1), (old_route2_index, new_route_2) = possible_change

            #print('modifying:', old_route1_index, old_route2_index)
            #print('from:', routes[old_route1_index], ' and:', routes[old_route2_index])
            #print('to:', new_route_1, 'and: ', new_route_2)

            routes[old_route1_index] = new_route_1
            routes[old_route2_index] = new_route_2

            new_cost = previous_cost - improvement
            
    return -new_cost

In [15]:
files = os.listdir("Instances")
files.remove('info.txt')
files.sort()

for filename in files:
    with open('Instances/' + filename, 'r') as f:
        content = f.read()
        #print(content)

    start = time.time()
    #START OF CONSTRUCTION
    content = content.split('\n')[0:-1] #-1 because the last element is a newline, so we skip it

    n_of_customers = int(content[0])
    n_of_vehicles = int(content[2])
    capacity = int(content[3].split()[3])
    linehaul_vector = [0] + [int(i.split()[2]) for i in content[4:]] #linehaul data is in the fourth column
    backhaul_vector = [0] + [int(i.split()[3]) for i in content[4:]] #and backhaul is in the third

    cost_matrix = get_cost_matrix()
    savings_matrix = get_savings_matrix ()
    routes = []
    for i in range(n_of_customers):
        routes += [(0, i+1, 0)]

    #Order savings
    ordered_savings = []

    for (i, j), el in np.ndenumerate(savings_matrix):
        ordered_savings += [(el, (i, j))]

    ordered_savings.sort(key=lambda x: x[0], reverse=True)

    total_cost = compute_total_cost()

    merge_routes()

    #Clean invalid routes
    new_routes = []

    for route in routes:

        if route != (-1, -1, -1):

            new_routes += [route]

    routes = new_routes
    constr_time = time.time() - start
    #END OF CONSTRUCTION   

    #BEST RELOCATE
    start = time.time()
    find_best_improvement('relocate')
    
    total_cost = compute_total_cost()
    rel_time = time.time() - start


    #BEST EXCHANGE
    start = time.time()
    find_best_improvement('exchange')
    
    total_cost = compute_total_cost()
    exc_time = time.time() - start
    
    print(routes)
    
    #LOG
    with open('parallel_routes/' + filename, 'w') as f:
        f.write('Total cost ' + str(total_cost))
        f.write('\nTime for construction ' + str(constr_time))
        f.write('\nTime for best relocate ' + str(rel_time))
        f.write('\nTime for best exchange ' + str(exc_time))
        f.write('\nTime for best improvement ' + str(rel_time+exc_time))
        for i in range(0,len(routes)):
            route = routes[i]
            f.write('\n\nRoute '+str(i))
            f.write('\n'+str(route))
            cost = 0
            for j in range(0,len(route)-1):
                cost += cost_matrix[j][j+1]
            f.write('\nCost ' + str(cost))

[[0, 63, 49, 34, 58, 123, 2, 14, 1, 0], [0, 50, 32, 3, 0], (0, 82, 26, 83, 5, 0), [0, 114, 121, 33, 43, 44, 68, 53, 12, 4, 7, 0], [0, 122, 74, 59, 99, 107, 28, 60, 109, 92, 27, 124, 80, 113, 64, 104, 18, 15, 8, 0], [0, 66, 120, 95, 37, 42, 56, 79, 24, 10, 0], [0, 48, 40, 115, 90, 91, 116, 100, 70, 78, 71, 96, 108, 77, 54, 57, 47, 61, 19, 25, 11, 0], (0, 39, 119, 30, 110, 86, 52, 36, 65, 13, 0), [0, 101, 84, 35, 51, 106, 97, 81, 16, 0], (0, 85, 31, 6, 17, 0), (0, 76, 89, 72, 87, 62, 45, 67, 117, 20, 9, 21, 0), (0, 111, 46, 103, 112, 22, 0), (0, 102, 125, 118, 69, 75, 88, 55, 94, 93, 38, 105, 73, 41, 98, 29, 23, 0)]
[[0, 142, 127, 112, 145, 87, 40, 75, 30, 20, 45, 3, 0], [0, 146, 79, 86, 102, 19, 4, 0], [0, 139, 76, 132, 131, 147, 116, 106, 135, 63, 13, 7, 66, 31, 16, 0], [0, 122, 84, 118, 94, 140, 77, 96, 95, 56, 11, 23, 0], (0, 85, 115, 81, 119, 117, 59, 32, 35, 34, 15, 25, 0), [0, 141, 78, 103, 89, 93, 46, 14, 73, 28, 47, 21, 38, 44, 0], (0, 114, 107, 113, 10, 60, 1, 2, 68, 65, 41, 27

[(0, 40, 63, 115, 90, 49, 34, 61, 58, 123, 2, 14, 1, 0), [0, 101, 76, 89, 30, 36, 52, 65, 3, 13, 0], (0, 82, 26, 83, 5, 0), [0, 122, 74, 114, 121, 33, 43, 44, 68, 53, 12, 4, 7, 0], (0, 48, 59, 99, 107, 28, 60, 109, 92, 27, 124, 80, 113, 64, 104, 18, 15, 8, 0), [0, 66, 120, 95, 37, 42, 50, 32, 56, 79, 24, 10, 0], [0, 91, 116, 100, 70, 78, 71, 96, 108, 77, 54, 57, 47, 19, 25, 11, 0], [0, 39, 119, 86, 110, 0], [0, 102, 125, 118, 69, 75, 88, 106, 51, 97, 81, 16, 0], (0, 85, 31, 6, 17, 0), [0, 84, 35, 72, 87, 62, 45, 67, 117, 20, 9, 21, 0], (0, 111, 46, 103, 112, 22, 0), (0, 55, 94, 93, 38, 105, 73, 41, 98, 29, 23, 0)]
[[0, 142, 127, 112, 145, 87, 40, 75, 30, 20, 45, 3, 0], [0, 146, 79, 86, 102, 19, 4, 0], [0, 139, 76, 132, 131, 147, 116, 106, 135, 63, 13, 7, 66, 31, 16, 0], [0, 122, 84, 118, 94, 140, 77, 96, 95, 56, 11, 23, 0], (0, 85, 115, 81, 119, 117, 59, 32, 35, 34, 15, 25, 0), [0, 141, 78, 103, 89, 93, 46, 14, 73, 28, 47, 21, 38, 44, 0], (0, 114, 107, 113, 10, 60, 1, 2, 68, 65, 41, 27

[[0, 19, 43, 54, 53, 57, 32, 14, 29, 37, 23, 39, 42, 2, 0], [0, 46, 35, 27, 33, 38, 48, 22, 1, 12, 3, 0], [0, 16, 41, 56, 25, 18, 52, 34, 15, 13, 20, 44, 50, 49, 47, 11, 8, 0], [0, 40, 26, 51, 24, 21, 31, 55, 45, 30, 36, 28, 17, 5, 10, 6, 4, 7, 9, 0]]
[(0, 9, 24, 20, 22, 1, 0), (0, 11, 15, 2, 0), [0, 8, 13, 25, 12, 10, 3, 0], (0, 16, 21, 14, 4, 0), (0, 17, 5, 0), (0, 18, 7, 6, 19, 23, 0)]
[(0, 40, 63, 115, 90, 49, 34, 61, 58, 123, 2, 14, 1, 0), [0, 101, 76, 89, 30, 36, 52, 65, 3, 13, 0], (0, 82, 26, 83, 5, 0), [0, 122, 74, 114, 121, 33, 43, 44, 68, 53, 12, 4, 7, 0], (0, 48, 59, 99, 107, 28, 60, 109, 92, 27, 124, 80, 113, 64, 104, 18, 15, 8, 0), [0, 66, 120, 95, 37, 42, 50, 32, 56, 79, 24, 10, 0], [0, 91, 116, 100, 70, 78, 71, 96, 108, 77, 54, 57, 47, 19, 25, 11, 0], [0, 39, 119, 86, 110, 0], [0, 102, 125, 118, 69, 75, 88, 106, 51, 97, 81, 16, 0], (0, 85, 31, 6, 17, 0), [0, 84, 35, 72, 87, 62, 45, 67, 117, 20, 9, 21, 0], (0, 111, 46, 103, 112, 22, 0), (0, 55, 94, 93, 38, 105, 73, 41, 98

In [29]:
#DEBUG

for route in routes:
    if route == (-1,-1,-1):
        continue
    total_l = 0
    total_b = 0
    for el in route:
        if el == 0:
            continue
        
        total_l += linehaul_vector[el]
        total_b += backhaul_vector[el]
    print(route)
    print(total_l)
    print(total_b)

[0, 63, 49, 34, 58, 123, 2, 14, 1, 0]
2899
1440
[0, 50, 32, 3, 0]
639
745
(0, 82, 26, 83, 5, 0)
1424
311
[0, 114, 121, 33, 43, 44, 68, 53, 12, 4, 7, 0]
4186
1618
[0, 122, 74, 59, 99, 107, 28, 60, 109, 92, 27, 124, 80, 113, 64, 104, 18, 15, 8, 0]
5940
1583
[0, 66, 120, 95, 37, 42, 56, 79, 24, 10, 0]
3251
1055
[0, 48, 40, 115, 90, 91, 116, 100, 70, 78, 71, 96, 108, 77, 54, 57, 47, 61, 19, 25, 11, 0]
7420
1354
(0, 39, 119, 30, 110, 86, 52, 36, 65, 13, 0)
4714
578
[0, 101, 84, 35, 51, 106, 97, 81, 16, 0]
3945
427
(0, 85, 31, 6, 17, 0)
1564
859
(0, 76, 89, 72, 87, 62, 45, 67, 117, 20, 9, 21, 0)
4566
1476
(0, 111, 46, 103, 112, 22, 0)
2058
408
(0, 102, 125, 118, 69, 75, 88, 55, 94, 93, 38, 105, 73, 41, 98, 29, 23, 0)
7598
576


In [30]:
#DEBUG
for route in routes:
    
    
    if route != (-1, -1, -1):
        
        print ('Route:')
        print (route)
        
        for item in route[1:-1]:
            
            if linehaul_vector[item] != 0:
                print('L')
                
            if backhaul_vector[item] != 0:
                print('B')
                

Route:
[0, 63, 49, 34, 58, 123, 2, 14, 1, 0]
L
L
L
L
L
B
B
B
Route:
[0, 50, 32, 3, 0]
L
L
B
Route:
(0, 82, 26, 83, 5, 0)
L
L
L
B
Route:
[0, 114, 121, 33, 43, 44, 68, 53, 12, 4, 7, 0]
L
L
L
L
L
L
L
B
B
B
Route:
[0, 122, 74, 59, 99, 107, 28, 60, 109, 92, 27, 124, 80, 113, 64, 104, 18, 15, 8, 0]
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
B
B
B
Route:
[0, 66, 120, 95, 37, 42, 56, 79, 24, 10, 0]
L
L
L
L
L
L
L
B
B
Route:
[0, 48, 40, 115, 90, 91, 116, 100, 70, 78, 71, 96, 108, 77, 54, 57, 47, 61, 19, 25, 11, 0]
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
B
B
B
Route:
(0, 39, 119, 30, 110, 86, 52, 36, 65, 13, 0)
L
L
L
L
L
L
L
L
B
Route:
[0, 101, 84, 35, 51, 106, 97, 81, 16, 0]
L
L
L
L
L
L
L
B
Route:
(0, 85, 31, 6, 17, 0)
L
L
B
B
Route:
(0, 76, 89, 72, 87, 62, 45, 67, 117, 20, 9, 21, 0)
L
L
L
L
L
L
L
L
B
B
B
Route:
(0, 111, 46, 103, 112, 22, 0)
L
L
L
L
B
Route:
(0, 102, 125, 118, 69, 75, 88, 55, 94, 93, 38, 105, 73, 41, 98, 29, 23, 0)
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
B


In [31]:
#DEBUG
for route in routes:
    if route == (-1,-1,-1):
        continue
    total_l = 0
    total_b = 0
    for el in route:
        if el == 0:
            continue
        
        total_l += linehaul_vector[el]
        total_b += backhaul_vector[el]
    print(route)
    print(total_l)
    print(total_b)

[0, 63, 49, 34, 58, 123, 2, 14, 1, 0]
2899
1440
[0, 50, 32, 3, 0]
639
745
(0, 82, 26, 83, 5, 0)
1424
311
[0, 114, 121, 33, 43, 44, 68, 53, 12, 4, 7, 0]
4186
1618
[0, 122, 74, 59, 99, 107, 28, 60, 109, 92, 27, 124, 80, 113, 64, 104, 18, 15, 8, 0]
5940
1583
[0, 66, 120, 95, 37, 42, 56, 79, 24, 10, 0]
3251
1055
[0, 48, 40, 115, 90, 91, 116, 100, 70, 78, 71, 96, 108, 77, 54, 57, 47, 61, 19, 25, 11, 0]
7420
1354
(0, 39, 119, 30, 110, 86, 52, 36, 65, 13, 0)
4714
578
[0, 101, 84, 35, 51, 106, 97, 81, 16, 0]
3945
427
(0, 85, 31, 6, 17, 0)
1564
859
(0, 76, 89, 72, 87, 62, 45, 67, 117, 20, 9, 21, 0)
4566
1476
(0, 111, 46, 103, 112, 22, 0)
2058
408
(0, 102, 125, 118, 69, 75, 88, 55, 94, 93, 38, 105, 73, 41, 98, 29, 23, 0)
7598
576


In [32]:
#DEBUG
for route in routes:
    
    
    if route != (-1, -1, -1):
        
        print ('Route:')
        print (route)
        
        for item in route[1:-1]:
            
            if linehaul_vector[item] != 0:
                print('L')
                
            if backhaul_vector[item] != 0:
                print('B')


Route:
[0, 63, 49, 34, 58, 123, 2, 14, 1, 0]
L
L
L
L
L
B
B
B
Route:
[0, 50, 32, 3, 0]
L
L
B
Route:
(0, 82, 26, 83, 5, 0)
L
L
L
B
Route:
[0, 114, 121, 33, 43, 44, 68, 53, 12, 4, 7, 0]
L
L
L
L
L
L
L
B
B
B
Route:
[0, 122, 74, 59, 99, 107, 28, 60, 109, 92, 27, 124, 80, 113, 64, 104, 18, 15, 8, 0]
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
B
B
B
Route:
[0, 66, 120, 95, 37, 42, 56, 79, 24, 10, 0]
L
L
L
L
L
L
L
B
B
Route:
[0, 48, 40, 115, 90, 91, 116, 100, 70, 78, 71, 96, 108, 77, 54, 57, 47, 61, 19, 25, 11, 0]
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
B
B
B
Route:
(0, 39, 119, 30, 110, 86, 52, 36, 65, 13, 0)
L
L
L
L
L
L
L
L
B
Route:
[0, 101, 84, 35, 51, 106, 97, 81, 16, 0]
L
L
L
L
L
L
L
B
Route:
(0, 85, 31, 6, 17, 0)
L
L
B
B
Route:
(0, 76, 89, 72, 87, 62, 45, 67, 117, 20, 9, 21, 0)
L
L
L
L
L
L
L
L
B
B
B
Route:
(0, 111, 46, 103, 112, 22, 0)
L
L
L
L
B
Route:
(0, 102, 125, 118, 69, 75, 88, 55, 94, 93, 38, 105, 73, 41, 98, 29, 23, 0)
L
L
L
L
L
L
L
L
L
L
L
L
L
L
L
B
